# Tidy Unshoveled Sidewalk Fines
3/2/24

This notebook organizes and tidies FOIA-requested datasets provided by the Department of Administrative Hearings:
1) One dataset of all court dockets for unshoveled sidewalks, including data on issuing department
2) One dataset of the same court dockets, including additional data on hearing date, case disposition, respondents, etc.

My goal is to join this into one comprehensive (albeit denormalized) dataset, dockets-to-details.csv
and three normalized datasets:

Though a full year of data is provided for the 2018-2019 season, I chose to look only at July 2019- June 2023 to align with data available for 311 complaints as well as City Council's legislative session running from May 2019 to May 2023.<br>

### summary of recordsets

Following is a summary of all recordsets prepared in this notebook, along with a count after filtering for 7/1/19 to 6/30/23

|recordset|num records|cardinality |description|
|---|---|---|---|
| dockets-to-addresses.csv | 1918 |  1:1 |docket numbers and cleaned addresses<br>to facilitate geocoding |
| dockets-to-depts.csv | 1917 |  1:1 |docket numbers to departments,<br>one record with missing dept |
| dockets-to-respondents.csv | 2240 | 1:many | one record per docket per respondent |
| dockets-detailed.csv | 3662 | 1:many | full details for each docket |

### notes on data quality
* one docket had no violation date (21DT00019A). Rather than estimate this date, I filtered it out
* one docket had no info about which department issued the citation. I retained this in dockets-to-details

The following dockets could not be geocoded:
* four dockets have address listed as UNKNOWN. This was not discovered until the geocoding phase
* one docket contains a nonsensical address- 5450 S 47TH

### overview of this data preparation
<ol>
<li>Read Data
<li>Prep Dockets Data (Simple)
<li>Prep Dockets Data (Detailed)
<li>Normalize Data
<li>Prep Addresses to Facilitate Geocoding
<li>Merge Department Info into Detailed Dockets Dataset
<li>Export Data
</ol>

# Read Data

In [1]:
import pandas as pd

In [2]:
df_dockets = pd.read_csv("../../data/00-raw/FOIA-H064920-011124-snow-dockets.csv")
df_dockets_detailed = pd.read_csv("../../data/00-raw/FOIA-H067483-021424-snow-dockets-detailed.csv")
df_corrections = pd.read_csv("../../data/00-raw/address-corrections.csv")

# Prep Dockets Data (Simple)

### read and review simple dockets dataset

In [3]:
df_dockets.head()

,Docket Number,Violation Date,Violation Address,Issuing Department Code,Imposed Fine Detailed
0,22CS00058A,02/04/2022,6621 S LANGLEY,BAFCONP,$0.00
1,22CS00059A,01/04/2022,6613 S LANGLEY AVE,BAFCONP,$0.00
2,18CP093079,11/27/2018,5045 N TRIPP,POLICE,$0.00
3,19CP003510,01/16/2019,5045 N TRIPP,POLICE,$0.00
4,19CP004491,01/06/2019,1800 S LAKE SHORE DR,POLICE,$0.00


In [4]:
len(df_dockets)

3058

In [5]:
df_dockets = df_dockets.rename(columns={'Docket Number':'docket','Violation Date':'violation_date',
                                        'Issuing Department Code':'dept',
                                        'Violation Address':'violation_address','Imposed Fine Detailed':'fine_amt'})
df_dockets.head()

,docket,violation_date,violation_address,dept,fine_amt
0,22CS00058A,02/04/2022,6621 S LANGLEY,BAFCONP,$0.00
1,22CS00059A,01/04/2022,6613 S LANGLEY AVE,BAFCONP,$0.00
2,18CP093079,11/27/2018,5045 N TRIPP,POLICE,$0.00
3,19CP003510,01/16/2019,5045 N TRIPP,POLICE,$0.00
4,19CP004491,01/06/2019,1800 S LAKE SHORE DR,POLICE,$0.00


### identify any dockets with missing violation date, which will be filtered out

In [6]:
df_dockets[df_dockets['violation_date'].isna()]

,docket,violation_date,violation_address,dept,fine_amt
1472,21DT00019A,NaN,1601 E 95TH ST,TRANPORT,$0.00
1473,21DT00019A,NaN,1601 E 95TH ST,TRANPORT,$150.00


### filter for 7/1/19 to 6/30/23

In [7]:
#convert violation date to datetime, so it can be filtered
df_dockets['violation_date'] = pd.to_datetime(df_dockets['violation_date'])

In [8]:
df_dockets = df_dockets[(df_dockets['violation_date']>='7/1/19') & (df_dockets['violation_date']<='6/30/23')]

In [9]:
# get number of unique dockets, for reference
df_dockets['docket'].nunique()

1917

# Prep Dockets Data (Detailed)

In [10]:
df_dockets_detailed.head()

,Respondent Full Name,Docket Number,Nov #,Violation Date,Hearing Date,Violation Address,Violation Description,Case Disposition Label,Imposed Fine Detailed
0,1010 W MADISON LLC,23DT000582,T000215159,01/31/2023,7/10/2023,1010 W MADISON,10-8-180 Snow and ice removal.,Non-Suit,$0.00
1,10-18 S HOMAN AVE HOLDINGS LLC c/o BRIAN GOTTE...,19DS41774L,241774L,01/22/2019,6/3/2019,10 S HOMAN AVE,10-8-180 Snow and ice removal.,Liable,$0.00
2,10-18 S HOMAN AVE HOLDINGS LLC c/o EV HAS LLC,19DS41774L,241774L,01/22/2019,6/3/2019,10 S HOMAN AVE,10-8-180 Snow and ice removal.,Liable,$0.00
3,"1100 E 47TH STREET LLC, C/O PETER CASSEL",22DT001042,T000207342,01/04/2022,5/23/2022,1100 E 47TH ST,10-8-180 Snow and ice removal.,Non-Suit,$0.00
4,"1101 BERWYN LLC, C/O JOHN BARRETT",23DT000559,T000220112,02/02/2023,3/20/2023,1101 W BERWYN,10-8-180 Snow and ice removal.,Non-Suit,$0.00


In [11]:
len(df_dockets_detailed)

4330

In [12]:
# simplify field names
df_dockets_detailed = df_dockets_detailed.rename(columns={'Respondent Full Name':'respondent',
                                                          'Docket Number':'docket',
                                                         'Nov #':'violation_num',
                                                         'Violation Date':'violation_date',
                                                         'Hearing Date':'hearing_date',
                                                         'Violation Address':'violation_address',
                                                         'Violation Description':'violation_desc',
                                                         'Case Disposition Label':'case_disposition',
                                                         'Imposed Fine Detailed':'fine_amt'})
df_dockets_detailed.head()

,respondent,docket,violation_num,violation_date,hearing_date,violation_address,violation_desc,case_disposition,fine_amt
0,1010 W MADISON LLC,23DT000582,T000215159,01/31/2023,7/10/2023,1010 W MADISON,10-8-180 Snow and ice removal.,Non-Suit,$0.00
1,10-18 S HOMAN AVE HOLDINGS LLC c/o BRIAN GOTTE...,19DS41774L,241774L,01/22/2019,6/3/2019,10 S HOMAN AVE,10-8-180 Snow and ice removal.,Liable,$0.00
2,10-18 S HOMAN AVE HOLDINGS LLC c/o EV HAS LLC,19DS41774L,241774L,01/22/2019,6/3/2019,10 S HOMAN AVE,10-8-180 Snow and ice removal.,Liable,$0.00
3,"1100 E 47TH STREET LLC, C/O PETER CASSEL",22DT001042,T000207342,01/04/2022,5/23/2022,1100 E 47TH ST,10-8-180 Snow and ice removal.,Non-Suit,$0.00
4,"1101 BERWYN LLC, C/O JOHN BARRETT",23DT000559,T000220112,02/02/2023,3/20/2023,1101 W BERWYN,10-8-180 Snow and ice removal.,Non-Suit,$0.00


### filter detailed dockets for 7/1/19 to 6/30/23

In [13]:
#convert violation date to datetime, so it can be filtered
df_dockets_detailed['violation_date'] = pd.to_datetime(df_dockets_detailed['violation_date'])

In [14]:
df_dockets_detailed = df_dockets_detailed[(df_dockets_detailed['violation_date']>='7/1/19') 
                                          & (df_dockets_detailed['violation_date']<='6/30/23')]

In [15]:
len(df_dockets_detailed)

3662

### identify data quality issues
Observation: the detailed docket dataset contains one docket (24DT000388) which is not assigned to any department

In [16]:
df_dockets_test_merge = pd.merge(df_dockets,df_dockets_detailed,on = "docket",how="right")

In [17]:
df_dockets_weird = df_dockets_test_merge[df_dockets_test_merge['dept'].isna()]
df_dockets_weird

,docket,violation_date_x,violation_address_x,dept,fine_amt_x,respondent,violation_num,violation_date_y,hearing_date,violation_address_y,violation_desc,case_disposition,fine_amt_y
6230,24DT000388,NaT,NaN,NaN,NaN,CTLTC 008002384127,T000215624,2023-01-17,5/6/2024,359 N CANAL ST,10-8-180 Snow and ice removal.,Pending,$0.00


### clean up fines data (clearing out $ helps with analysis)

In [18]:
# clean up fine data
df_dockets_detailed['fine_amt'] = pd.to_numeric(df_dockets_detailed['fine_amt'].replace('[\$,]', '', regex=True), errors='coerce')

In [19]:
# get number of unique dockets, for reference
df_dockets_detailed['docket'].nunique()

1918

# Normalize Data

### dockets to departments
1917 records, because one record is missing department

In [20]:
df_dockets_to_depts = df_dockets[['docket','dept']].drop_duplicates()

In [21]:
df_dockets_to_depts.head()

,docket,dept
0,22CS00058A,BAFCONP
1,22CS00059A,BAFCONP
63,20CP00166A,POLICE
65,20CP00167A,POLICE
66,20CP005520,POLICE


In [22]:
len(df_dockets_to_depts)

1917

### dockets to respondents
one docket can have multiple respondents. remove this for now as it could be calculated downstream during the data analysis phase

In [23]:
#df_dockets_to_respondents = df_dockets_detailed[['docket','respondent']].drop_duplicates()

In [24]:
#len(df_dockets_to_respondents)

### dockets to addresses

In [25]:
df_dockets_to_addresses = df_dockets_detailed[['docket','violation_address']].drop_duplicates()

In [26]:
df_dockets_to_addresses.head()

,docket,violation_address
0,23DT000582,1010 W MADISON
3,22DT001042,1100 E 47TH ST
4,23DT000559,1101 W BERWYN
5,22DT001387,100 UNKNOWN
6,21DS20339M,1200 W BELMONT AVE


In [27]:
len(df_dockets_to_addresses)

1918

# Prep Addresses for Geocoding

### clean up addresses

In [28]:
# add city and state, which are necessary for geocoding
df_dockets_to_addresses['city']='Chicago'
df_dockets_to_addresses['state']='Illinois'

In [29]:
# strip out trailing whitespace to get address merge to function. otherwise about 75% don't link correctly
df_dockets_to_addresses['violation_address'] = df_dockets_to_addresses['violation_address'].str.strip()

### merge with address corrections
I created the dataset "address-corrections" manually based on addresses which failed to geocode

In [30]:
df_corrections.head()

,violation_address,cleaned_address,corrected
0,10320 S CHURCH,10320 S CHURCH ST,True
1,1108 W GRAND,1108 W GRAND AVE,True
2,1111 W MARSHFIELD,1111 N MARSHFIELD,True
3,1146 W GRAND,1146 W GRAND AVE,True
4,115 W LAWERENCE,115 W LAWRENCE,True


In [31]:
df_dockets_to_addresses_clean = pd.merge(df_dockets_to_addresses,df_corrections, on='violation_address', how='left')

In [32]:
# for correct records, copy existing address to cleaned address 
df_dockets_to_addresses_clean.loc[
    df_dockets_to_addresses_clean['corrected'].isna(), 'cleaned_address'] = df_dockets_to_addresses_clean['violation_address']
df_dockets_to_addresses_clean.head()

,docket,violation_address,city,state,cleaned_address,corrected
0,23DT000582,1010 W MADISON,Chicago,Illinois,1010 W MADISON,NaN
1,22DT001042,1100 E 47TH ST,Chicago,Illinois,1100 E 47TH ST,NaN
2,23DT000559,1101 W BERWYN,Chicago,Illinois,1101 W BERWYN,NaN
3,22DT001387,100 UNKNOWN,Chicago,Illinois,100 UNKNOWN,NaN
4,21DS20339M,1200 W BELMONT AVE,Chicago,Illinois,1200 W BELMONT AVE,NaN


In [33]:
len(df_dockets_to_addresses_clean)

1918

In [34]:
df_dockets_to_addresses_clean.groupby('corrected').size()

corrected
True    88
dtype: int64

In [35]:
# sample some cleaned addresses
df_dockets_to_addresses_clean[df_dockets_to_addresses_clean['corrected']==1].head()

,docket,violation_address,city,state,cleaned_address,corrected
11,21DT000514,164 E GRAND,Chicago,Illinois,164 E GRAND AVE,True
55,20DT001518,6815 W ARCHERS,Chicago,Illinois,6815 W ARCHER,True
56,20DT001519,5501 S NEW CASTLE,Chicago,Illinois,5501 S NEWCASTLE,True
62,20DT004074,3956 W GRAND,Chicago,Illinois,3956 W GRAND AVE,True
71,20DT001526,5348 S CENTRAL,Chicago,Illinois,5348 S CENTRAL AVE,True


# Merge Department Info into Detailed Dockets Dataset
This gets all useful info in one place, albeit in denormalized form

In [36]:
len(df_dockets_detailed)

3662

In [37]:
# left merge, to hold onto the record without a department assignment
df_dockets_detailed_dept = pd.merge(df_dockets_detailed,df_dockets_to_depts, on = 'docket', how = "left")
df_dockets_detailed_dept.head()

,respondent,docket,violation_num,violation_date,hearing_date,violation_address,violation_desc,case_disposition,fine_amt,dept
0,1010 W MADISON LLC,23DT000582,T000215159,2023-01-31,7/10/2023,1010 W MADISON,10-8-180 Snow and ice removal.,Non-Suit,0.0,TRANPORT
1,"1100 E 47TH STREET LLC, C/O PETER CASSEL",22DT001042,T000207342,2022-01-04,5/23/2022,1100 E 47TH ST,10-8-180 Snow and ice removal.,Non-Suit,0.0,TRANPORT
2,"1101 BERWYN LLC, C/O JOHN BARRETT",23DT000559,T000220112,2023-02-02,3/20/2023,1101 W BERWYN,10-8-180 Snow and ice removal.,Non-Suit,0.0,TRANPORT
3,1157 W ERIE,22DT001387,T000207431,2022-02-08,4/11/2022,100 UNKNOWN,10-8-180 Snow and ice removal.,Not Liable,0.0,TRANPORT
4,1200B INC C/O THOMAS GEORGE,21DS20339M,QTUF62L,2021-02-05,7/14/2021,1200 W BELMONT AVE,10-8-180 Snow and ice removal.,Not Liable,0.0,STRTSAN


In [38]:
len(df_dockets_detailed_dept)

3662

In [39]:
df_dockets_detailed_dept['docket'].nunique()

1918

# Export Data

In [40]:
df_dockets_to_depts.to_csv('../../data/01-tidied/dockets-to-depts.csv', index= False)
df_dockets_to_addresses_clean.to_csv('../../data/01-tidied/dockets-to-addresses.csv', index = False)
df_dockets_detailed_dept.to_csv('../../data/01-tidied/dockets-to-details.csv', index = False)
#df_dockets_to_respondents.to_csv('../../data/01-tidied/dockets-to-respondents.csv', index = False)